In [1]:
import whisper
import os
from pathlib import Path  
import numpy as np 

In [2]:


model = whisper.load_model(name='large-v2',download_root="/media/checkpoint/whisper-large-v2-pt/")


In [9]:
import jiwer
from whisper.normalizers import BasicTextNormalizer
import pandas as pd
normalizer = BasicTextNormalizer()
data = pd.DataFrame()

In [33]:


result1 = model.transcribe(initial_prompt='以下是普通话的句子.',audio="/media/checkpoint/speech_data/0_gt_wavs/0_6.wav",language='zh')
print(result1)


result2 = model.transcribe(initial_prompt='以下是普通话的句子.',audio="/media/checkpoint/speech_data/0_gt_wavs/0_6.wav",language='zh')
print(result2) 



if(len(result1['segments'])>0):

    data["hypothesis_clean"] = [normalizer(result1['text']) ]
    data["reference_clean"] = [normalizer(result2['text'])]
    print(data)
    wer = jiwer.wer(list(data["reference_clean"]), list(data["hypothesis_clean"]))

    print(f"WER: {wer * 100:.2f} %")

{'text': '', 'segments': [], 'language': 'zh'}
{'text': '', 'segments': [], 'language': 'zh'}


In [199]:

result1 = model.transcribe(initial_prompt='以下是普通话的句子.',temperature=1,audio="/media/checkpoint/speech_data/1_16k_wavs/15_28.wav",language='zh')
print(result1)

result2 = model.transcribe(initial_prompt="以下是普通话的句子,无法识别,停止解析",temperature=0.1,audio="/media/checkpoint/speech_data/1_16k_wavs/15_28.wav",language='zh')
print(result2)



if(len(result1['segments'])>0):

    data["hypothesis_clean"] = [normalizer(result1['text']) ]
    data["reference_clean"] = [normalizer(result2['text'])]
    print(data)
    wer = jiwer.wer(list(data["reference_clean"]), list(data["hypothesis_clean"]))

    print(wer)

{'text': '', 'segments': [], 'language': 'zh'}
{'text': '请不吝点赞 订阅 转发 打赏支持明镜与点点栏目', 'segments': [{'id': 0, 'seek': 0, 'start': 0.0, 'end': 4.0, 'text': '请不吝点赞 订阅 转发 打赏支持明镜与点点栏目', 'tokens': [50364, 27908, 1960, 2392, 251, 12579, 5266, 252, 220, 7422, 95, 10034, 227, 220, 17819, 105, 28926, 220, 12467, 5266, 237, 35488, 11100, 12373, 250, 940, 236, 12579, 12579, 8987, 237, 11386, 50564], 'temperature': 0.1, 'avg_logprob': -0.22846506623660817, 'compression_ratio': 0.8513513513513513, 'no_speech_prob': 0.8216071128845215}], 'language': 'zh'}


In [200]:
'''
获取输入文件夹内的所有wav文件，并返回文件名全称列表
'''
def file_name(file_dir):
    L = []
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if os.path.splitext(file)[1] == '.wav':
                filename=os.path.join(root, file)
                L.append(filename)
        return L 

In [201]:
def transcribe_whisper(audio):
    data = {}
    data['audio']=audio
    result1 = model.transcribe(initial_prompt='以下是普通话的句子.', temperature=1, audio=audio,language='zh') 
    result2 = model.transcribe(initial_prompt="以下是普通话的句子,无法识别,停止解析",temperature=0.1, audio=audio,language='zh') 
    if(len(result1['text'].strip())>0 and len(result1['text'].strip())>0):
        data["hypothesis_clean"] = result1['text']
        data["reference_clean"] = result2['text'] 
        normalizer1 = normalizer(result1['text'])
        normalizer2 = normalizer(result2['text'])
        try:
            data["WER"] = jiwer.wer(list([normalizer1] ), list( [normalizer2 ])) 
        except:
            data["WER"] = 1
            
    return data;

In [202]:
def batch(iterable, size):
    # range对象的step是size
    for i in range(0, len(iterable), size):
        yield iterable[i:i + size]

In [203]:
import shutil
from tqdm import tqdm
import time
wav_dir="/media/checkpoint/speech_data/1_16k_wavs/"#语音文件夹
output_wav_dir="/media/checkpoint/speech_data/output/"#输出语音文件夹

ds_path = Path(output_wav_dir)
if ds_path.exists() is False:
    ds_path.mkdir()
print("获取数据")
wav_files=file_name(wav_dir)#获取文件夹内的所有语音文件
print("获取数据，成功{}".format(len(wav_files)))
#对每一个文件进行操作
batch_list = batch(wav_files,10)
b_unit = tqdm(enumerate(wav_files),total=len(wav_files))
for batch_file in batch_list:  
    
    table_data=[]
    for filename in batch_file:
        time.sleep(1)
        data = transcribe_whisper(filename)  
        table_data.append(data)
    
    table = pd.DataFrame(table_data, columns=["audio","hypothesis_clean", "reference_clean", "WER"])     

    filtered_rows = table[table['WER'] <0.1]    
    aidatatang_200_zh_transcript_text = []
    for row in filtered_rows.itertuples():
        #判断文件夹是否存在，不存在创建 
        f_name, file_ext = os.path.splitext(os.path.basename(row.audio) ) 
        aidatatang_200_zh_transcript_text.append(f_name+' '+row.hypothesis_clean)
        
    with open('/media/checkpoint/speech_data/aidatatang_200_zh_transcript.txt', mode='a') as filename:
        filename.write('\r\n'.join(aidatatang_200_zh_transcript_text))

    for row in filtered_rows.itertuples():
        # 判断文件夹是否存在，不存在创建 
        f_name, file_ext = os.path.splitext(os.path.basename(row.audio) ) 
         
        sr_fname = f"{wav_dir}{f_name}{ext}"
        ds_fname = f"{output_wav_dir}{f_name}{ext}"
        # Copy src to dst. (cp src dst)
        shutil.copy(sr_fname, ds_fname)

    # 更新进度
    b_unit.update(10)

            

获取数据
获取数据，成功7046



  9%|███▉                                         | 610/7046 [32:22<5:41:40,  3.19s/it]

100%|███████████████████████████████████████████▉| 7040/7046 [4:11:22<00:12,  2.11s/it]
7050it [4:11:36,  1.90s/it]                                                            